<a href="https://colab.research.google.com/github/molabokchi/bokchi_open_lab/blob/main/event_dog_bucket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## reinstall diffusers 0.2.4
## TODO: you should restart kernel

#!pip install diffusers==0.8.0

#! pip install diffusers==0.14.0

In [ ]:

! pip --version
! python --version
! nvidia-smi


In [ ]:
!pip install diffusers==0.14.0
#!pip install transformers scipy ftfy accelerate
!pip install transformers scipy ftfy accelerate

import torch
from diffusers import StableDiffusionPipeline


In [ ]:
! rm -rf diffusers
! git clone https://github.com/huggingface/diffusers
! cd diffusers && git checkout v0.14.0 && pip install -e .


In [ ]:
! cd diffusers/examples/dreambooth && pip install -r requirements.txt


In [ ]:
! accelerate config default


In [ ]:
from accelerate.utils import write_basic_config
!rm ~/.cache/huggingface/accelerate/default_config.yaml
write_basic_config()

In [ ]:
! pip install bitsandbytes

In [ ]:
! pip install -U xformers

In [ ]:
! curl -sSL -O https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/instance-images.tar.gz

!tar -xzvf instance-images.tar.gz

! rm -rf ./instance-images/.ipynb_checkpoints


In [ ]:
### 12GB GPU

#export MODEL_NAME="CompVis/stable-diffusion-v1-4"
#export INSTANCE_DIR="path-to-instance-images"
#export CLASS_DIR="path-to-class-images"
#export OUTPUT_DIR="path-to-save-model"

! accelerate launch ./diffusers/examples/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4  \
  --instance_data_dir=./instance-images \
  --class_data_dir=./class_dir \
  --output_dir=./output \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of sks dog" \
  --class_prompt="a photo of dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --use_8bit_adam \
  --enable_xformers_memory_efficient_attention \
  --set_grads_to_none \
  --learning_rate=2e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=800

In [ ]:
## 16GB GPU



# export MODEL_NAME="CompVis/stable-diffusion-v1-4"
# export INSTANCE_DIR="path-to-instance-images"
# export CLASS_DIR="path-to-class-images"
# export OUTPUT_DIR="path-to-save-model"

# ! accelerate launch ./diffusers/examples/dreambooth/train_dreambooth.py \
#   --pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4  \
#   --instance_data_dir=/content/instance_image \
#   --class_data_dir=/content/class_dir \
#   --output_dir=/content/output \
#   --with_prior_preservation --prior_loss_weight=1.0 \
#   --instance_prompt="a photo of sks dog" \
#   --class_prompt="a photo of dog" \
#   --resolution=512 \
#   --train_batch_size=1 \
#   --gradient_accumulation_steps=2 --gradient_checkpointing \
#   --use_8bit_adam \
#   --learning_rate=5e-6 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --num_class_images=200 \
#   --max_train_steps=800

In [ ]:

## 8GPU, VRAM

# ! accelerate launch --mixed_precision="fp16" ./diffusers/examples/dreambooth/train_dreambooth.py \
#   --pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4 \
#   --instance_data_dir=/content/instance_image \
#   --class_data_dir=/content/class_dir \
#   --output_dir=/content/output \
#   --with_prior_preservation --prior_loss_weight=1.0 \
#   --instance_prompt="a photo of sks dog" \
#   --class_prompt="a photo of dog" \
#   --resolution=512 \
#   --train_batch_size=1 \
#   --sample_batch_size=1 \
#   --gradient_accumulation_steps=1 --gradient_checkpointing \
#   --learning_rate=5e-6 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --num_class_images=200 \
#   --max_train_steps=800

In [ ]:
# from diffusers import DiffusionPipeline, UNet2DConditionModel
# from transformers import CLIPTextModel
# import torch

# # Load the pipeline with the same arguments (model, revision) that were used for training
# model_id = "CompVis/stable-diffusion-v1-4"

# unet = UNet2DConditionModel.from_pretrained("/content/output/checkpoint-500/unet")

# # if you have trained with `--args.train_text_encoder` make sure to also load the text encoder
# text_encoder = CLIPTextModel.from_pretrained("/content/output/text_encoder")

# pipeline = DiffusionPipeline.from_pretrained(model_id, unet=unet, text_encoder=text_encoder, dtype=torch.float16)
# pipeline.to("cuda")

# # Perform inference, or save, or push to the hub
# pipeline.save_pretrained("dreambooth-pipeline")

In [ ]:
# ! pip install --upgrade diffusers transformers accelerate


from diffusers import StableDiffusionPipeline
import torch

model_id = "./output"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "A photo of sks dog in a bucket"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save("dog-bucket.png")
image

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

num_cols = 3
num_rows = 4

#prompt = ["a photograph of an astronaut riding a horse"] * num_cols
prompt = ["a photo of sks dog in a bucket"] * num_cols

all_images = []
for i in range(num_rows):
  images = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid